In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

### Make scene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.panda, None,
                INDY_IP)]
              , connection_list=[False])
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"panda0": ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

connection command:
panda0: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]


## Preapare background

In [3]:
from pkg.geometry.geometry import *
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.3,0,0), 
                           rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)

gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="panda0_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
grip0 = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
gscene.show_pose(crob.home_pose)

Please create a subscriber to the marker


## Door

In [4]:
from pkg.utils.code_scraps import *

In [5]:
door, door_s = add_door(pscene, center=(0.7,-0.2,0.5), color=(0.8,0.4,0.2,1))
knob = gscene.create_safe(GEOTYPE.BOX, 'knob', link_name="base_link",
                          center=(-door.dims[0]/2-0.01,-door.dims[1]/2+0.05,0), rpy=(0,-np.pi/2,0), dims=(0.02,0.02,0.02),
                          fixed=False, collision=True, color=(0.8,0.8,0.8,1), parent="door")

knob_s = add_lever(pscene, knob, dims=(0.02,0.1,0.02), knob_offset=0.04)

### prepare planner

In [6]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene, enable_dual=False)
mplan.update_gscene()

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

from pkg.ui.ui_broker import *
# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)


In [7]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
mplan.motion_filters = [GraspChecker(pscene), ReachChecker(pscene)]
mplan.incremental_constraint_motion = True

 * Environment: production


### Plan

In [8]:
gscene.show_pose(crob.home_pose)
initial_state = pscene.initialize_state(crob.home_pose)
print(pscene.subject_name_list)
print(initial_state.node)

['door', 'knob', 'lever_grip']
(0, 0, 'knob')


In [9]:
goal_nodes = [(2, 2,'grip0')]
ppline.search(initial_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=30, 
              multiprocess=True)

try: 0 - (0, 0, 'knob')->(0, 0, 'grip0')
try transition motion
transition motion tried: True
   Use a production WSGI server instead.
result: 0 - (0, 0, 'knob')->(0, 0, 'grip0') = success
branching: 0->1 (0.08/30.0 s, steps/err: 52(71.9108581543 ms)/0.00122737861866)
 * Debug mode: off
try: 1 - (0, 0, 'grip0')->(0, 1, 'grip0')
result: 1 - (0, 0, 'grip0')->(0, 1, 'grip0') = success
branching: 1->2 (0.09/30.0 s, steps/err: 1(5.31196594238 ms)/0)
try: 2 - (0, 1, 'grip0')->(1, 1, 'grip0')
result: 2 - (0, 1, 'grip0')->(1, 1, 'grip0') = fail
try: 1 - (0, 0, 'grip0')->(1, 0, 'grip0')
result: 1 - (0, 0, 'grip0')->(1, 0, 'grip0') = fail
try: 0 - (0, 0, 'knob')->(1, 0, 'knob')
result: 0 - (0, 0, 'knob')->(1, 0, 'knob') = fail
try: 0 - (0, 0, 'knob')->(0, 1, 'knob')
result: 0 - (0, 0, 'knob')->(0, 1, 'knob') = fail
try: 1 - (0, 0, 'grip0')->(1, 0, 'grip0')
result: 1 - (0, 0, 'grip0')->(1, 0, 'grip0') = fail
try: 2 - (0, 1, 'grip0')->(0, 2, 'grip0')
end
constrained motion tried: True
result: 2 - (

In [10]:
pscene.set_object_state(initial_state)
gscene.show_pose(initial_state.Q)

In [11]:
snode_schedule = tplan.get_best_schedule(at_home=False)
ppline.play_schedule(snode_schedule)

(0, 0, 'knob')->(0, 0, 'grip0')
(0, 0, 'grip0')->(0, 1, 'grip0')
(0, 1, 'grip0')->(0, 2, 'grip0')
(0, 2, 'grip0')->(1, 2, 'grip0')
(1, 2, 'grip0')->(2, 2, 'grip0')


### push back

In [12]:
term_state = snode_schedule[-1].state
pscene.set_object_state(term_state)
gscene.show_pose(term_state.Q)
print(pscene.subject_name_list)
print(term_state.node)

['door', 'knob', 'lever_grip']
(2, 2, 'grip0')


In [13]:
goal_nodes = [(1, 1,'knob')]
ppline.search(term_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=30, 
              multiprocess=True)

Use 18/36 agents
try: 0 - (2, 2, 'grip0')->(2, 2, 'knob')
try: 0 - (2, 2, 'grip0')->(2, 2, 'knob')
link transfer between same links: ['lever_grip'] - panda0
Motion Plan Failure
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
link transfer between same links: ['lever_grip'] - panda0
result: 0 - (2, 2, 'grip0')->(2, 2, 'knob') = fail
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
Motion Plan Failure
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
link transfer between same links: ['knob'] - panda0
link transfer between same links: ['knob'] - panda0
result: 0 - (2, 2, 'grip0')->(2, 2, 'knob') = fail
Motion Plan Failure
Motion Plan Failure
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
result: 0 - (2, 2, 'grip0')->(2, 1, 'grip0') = fail
result: 0 - (2, 2, 'grip0')->(2, 1, 'grip0') = fail
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
link transfer between same links: ['knob'] - panda0
Motion Plan Failure
try: 0 - (2, 2, 'grip0')->(2,

link transfer between same links: ['knob'] - panda0
Motion Plan Failure
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
result: 4 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
try: 1 - (1, 2, 'grip0')->(1, 2, 'grip0')
link transfer between same links: ['knob'] - panda0
Motion Plan Failure
end
constrained motion tried: True
result: 1 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
result: 0 - (2, 2, 'grip0')->(1, 2, 'grip0') = success
try: 4 - (1, 2, 'grip0')->(1, 2, 'grip0')
branching: 0->5 (0.98/30.0 s, steps/err: 81(626.899957657 ms)/0.000155540352465)
link transfer between same links: ['knob'] - panda0
end
Motion Plan Failure
try: 5 - (1, 2, 'grip0')->(1, 2, 'knob')
constrained motion tried: True
link transfer between same links: ['lever_grip'] - panda0
end
constrained motion tried: True
Motion Plan Failure
result: 0 - (2, 2, 'grip0')->(1, 2, 'grip0') = success
result: 4 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
branching: 0->6 (1.02/30.0 s, steps/err: 81(514.789104462 ms)/0.000155540352465)
try: 6

try: 16 - (1, 2, 'grip0')->(1, 2, 'grip0')
branching: 0->17 (1.38/30.0 s, steps/err: 81(762.611150742 ms)/0.000155540352465)
try: 17 - (1, 2, 'grip0')->(1, 2, 'knob')
Motion Plan Failure
try: 0 - (2, 2, 'grip0')->(2, 2, 'knob')
link transfer between same links: ['knob'] - panda0
result: 15 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
result: 9 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
link transfer between same links: ['lever_grip'] - panda0
Motion Plan Failure
link transfer between same links: ['lever_grip'] - panda0
try: 11 - (1, 2, 'grip0')->(1, 2, 'knob')
Motion Plan Failure
Motion Plan Failure
try: 15 - (1, 2, 'grip0')->(2, 2, 'grip0')
result: 16 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
link transfer between same links: ['lever_grip'] - panda0
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
Not available:door-0
result: 0 - (2, 2, 'grip0')->(2, 2, 'knob') = fail
Not available:door-0
np_exclude:['h0']
Motion Plan Failure
try: 3 - (1, 2, 'grip0')->(1, 2, 'knob')
np_exclude:['h0']
result: 11 

try: 16 - (1, 2, 'grip0')->(1, 2, 'grip0')
link transfer between same links: ['knob'] - panda0
Motion Plan Failure
result: 16 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
link transfer between same links: ['knob'] - panda0
Motion Plan Failure
result: 0 - (2, 2, 'grip0')->(2, 1, 'grip0') = fail
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
end
constrained motion tried: True
x
result: 1 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 1->23 (1.89/30.0 s, steps/err: 81(512.478113174 ms)/0.000128042839292)
end
constrained motion tried: True
try: 2 - (1, 2, 'grip0')->(1, 2, 'grip0')
x
result: 9 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 9->24 (1.9/30.0 s, steps/err: 81(648.100137711 ms)/0.000128042839292)
link transfer between same links: ['knob'] - panda0
Motion Plan Failure
end
constrained motion tried: True
result: 2 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
try: 24 - (2, 2, 'grip0')->(2, 2, 'grip0')
result: 0 - (2, 2, 'grip0')->(1, 

link transfer between same links: ['lever_grip'] - panda0
link transfer between same links: ['lever_grip'] - panda0
Motion Plan Failure
Motion Plan Failure
try: 24 - (2, 2, 'grip0')->(2, 2, 'grip0')
result: 30 - (2, 2, 'grip0')->(2, 2, 'knob') = fail
try: 22 - (1, 2, 'grip0')->(1, 2, 'knob')
link transfer between same links: ['knob'] - panda0
Motion Plan Failure
result: 0 - (2, 2, 'grip0')->(2, 2, 'knob') = fail
end
link transfer between same links: ['lever_grip'] - panda0
Motion Plan Failure
constrained motion tried: True
end
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
constrained motion tried: True
result: 22 - (1, 2, 'grip0')->(1, 2, 'knob') = fail
result: 24 - (2, 2, 'grip0')->(2, 1, 'grip0') = fail
end
result: 0 - (2, 2, 'grip0')->(1, 2, 'grip0') = success
try: 27 - (2, 2, 'grip0')->(2, 2, 'knob')
x
constrained motion tried: True
result: 9 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 0->33 (2.23/30.0 s, steps/err: 81(651.747941971 ms)/0.000155540352465)
Not available:door

...

try: 86 - (1, 2, 'grip0')->(2, 2, 'grip0')
============== Non-available transition: sample again =====================
link transfer between same links: ['lever_grip'] - panda0
Motion Plan Failure
result: 62 - (1, 2, 'grip0')->(1, 2, 'knob') = fail
try: 124 - (2, 2, 'grip0')->(2, 2, 'grip0')
try: 34 - (2, 2, 'grip0')->(2, 2, 'grip0')
link transfer between same links: ['knob'] - panda0
Motion Plan Failure
link transfer between same links: ['knob'] - panda0
Motion Plan Failure
result: 124 - (2, 2, 'grip0')->(2, 1, 'grip0') = fail
try: 135 - (1, 2, 'grip0')->(1, 2, 'knob')
result: 34 - (2, 2, 'grip0')->(2, 1, 'grip0') = fail
end
try: 16 - (1, 2, 'grip0')->(1, 2, 'knob')
constrained motion tried: True
link transfer between same links: ['lever_grip'] - panda0
end
x
constrained motion tried: True
link transfer between same links: ['lever_grip'] - panda0
end
result: 50 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
constrained motion tried: True
result: 89 - (2, 2, 'grip0')->(1, 2, 'grip0') = s

Not available:door-0
np_exclude:['h0']
bd_exclude:door_hinge
============== Non-available transition: sample again =====================
============== Non-available transition: sample again =====================
Not available:door-0
np_exclude:['h0']
bd_exclude:door_hinge
Not available:door-0
np_exclude:['h0']
bd_exclude:door_hinge
============== Non-available transition: sample again =====================
end
constrained motion tried: True
============== Non-available transition: sample again =====================
try: 6 - (1, 2, 'grip0')->(2, 2, 'grip0')
try: 115 - (1, 2, 'grip0')->(1, 2, 'knob')
result: 155 - (2, 2, 'grip0')->(1, 2, 'grip0') = success
branching: 155->170 (7.94/30.0 s, steps/err: 81(588.092088699 ms)/0.000139210451573)
link transfer between same links: ['lever_grip'] - panda0
Motion Plan Failure
try: 170 - (1, 2, 'grip0')->(1, 2, 'grip0')
link transfer between same links: ['knob'] - panda0
result: 115 - (1, 2, 'grip0')->(1, 2, 'knob') = fail
Motion Plan Failure
end


end
try: 163 - (1, 2, 'grip0')->(2, 2, 'grip0')
x
constrained motion tried: True
result: 22 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
x
branching: 22->177 (8.22/30.0 s, steps/err: 81(686.458826065 ms)/0.000128042839292)
result: 142 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
Not available:door-0
branching: 142->178 (8.22/30.0 s, steps/err: 81(704.078912735 ms)/0.00018326876969)
try: 104 - (2, 2, 'grip0')->(2, 2, 'grip0')
np_exclude:['h0']
bd_exclude:door_hinge
link transfer between same links: ['knob'] - panda0
============== Non-available transition: sample again =====================
Motion Plan Failure
Not available:door-0
result: 104 - (2, 2, 'grip0')->(2, 1, 'grip0') = fail
try: 119 - (1, 2, 'grip0')->(1, 2, 'knob')
np_exclude:['h0']
link transfer between same links: ['lever_grip'] - panda0
bd_exclude:door_hinge
Motion Plan Failure
Not available:door-0
np_exclude:['h0']
============== Non-available transition: sample again =====================
bd_exclude:door_hinge
result: 11

end
constrained motion tried: True
x
result: 93 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 93->190 (8.69/30.0 s, steps/err: 81(644.090890884 ms)/0.00018326876969)
end
constrained motion tried: True
x
result: 62 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 62->191 (8.71/30.0 s, steps/err: 81(569.213151932 ms)/0.00018326876969)
end
constrained motion tried: True
x
result: 117 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 117->192 (8.76/30.0 s, steps/err: 81(611.049890518 ms)/0.00018326876969)
end
constrained motion tried: True
x
end
result: 100 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
constrained motion tried: True
branching: 100->193 (8.82/30.0 s, steps/err: 81(508.264064789 ms)/0.00018326876969)
x
result: 90 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 90->194 (8.83/30.0 s, steps/err: 81(490.211009979 ms)/0.00018326876969)
end
constrained motion tried: True
x
result: 121 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 121->195 (8

======================= terminated 3: node queue empty 16th  (9.6/30.0) ===============================
======================= terminated 4: node queue empty 17th  (9.6/30.0) ===============================
======================= terminated 1: node queue empty 18th  (9.6/30.0) ===============================
========================== FINISHED (9.7 / 30.0 s) ==============================]


In [14]:
snode_schedule = tplan.get_best_schedule()
ppline.play_schedule(snode_schedule)

IndexError: list index out of range

generate table - Geometry
generate table - Handle
 generate table - Object
generate table - Binder
generate table - PlanConditions
generate table - PlanList
button clicked
(2, 2, 'grip0')->(1, 2, 'grip0')
button action done
